# 决策树算法原理及应用笔记
决策树算法在机器学习中算是一个很经典的算法系列了。     
它既可以作为分类算法， 也可以作为回归算法，同时也特别适合集成学习，比如随机森林。    
主要对ID3，C4.5，CART算法做一个详细的介绍。   
CART算法做重点介绍的原因是scikit-learn使用了优化版的CART算法作为其决策树算法的实现。

## 1. 决策树ID3算法的信息论基础
if-elif-else其实已经用到了决策树的思想。  
但是那么多的条件，用哪个条件特征先做if，哪个条件特征后做if比较优呢？   
**怎么准确的定量选择这个标准就是决策树机器学习算法的关键。**


1970年代，昆兰用**信息论**中的**熵**来度量决策树的决策选择过程，这个算法就是ID3.    

首先，我们需要熟悉信息论中熵的概念。    
**熵度量了事物的不确定性，越不确定的事物，它的熵就越大。**
公式：
    $H(X) = -\sum_{i=1}^{n}p_{i}log_{2}p_{i}$

其中，n代表X的n种不同的离散取值。   
而$p_{i}$代表了$X$取值为$i$的概率。  
log为以2或者e为底的对数。    


例如：一个样本中，总共4个实例，分别是2个1和2个0，那么这个样本的熵就是1。（计算过程见下方代码处，如还需其他的实例参考，<a href='../3DecisionTree.ipynb'>示例</a>


熟悉了一个变量$X$的熵，推广到多个变量的**联合熵**，这里给出两个变量$X$和$Y$的联合熵表达式：
    $H(X,Y)=-\sum_{i=1}^{n}p(x_{i},y_{i})log_{2}p(x_{i},y_{i})$
    

有了联合熵，又可以得到**条件熵**的表达式$H(X|Y)$，条件熵类似于条件概率，它度量了我们的$X$在知道$Y$以后剩下的不确定性。  
**条件熵的表达式**：
    $H(X|Y)=-\sum_{i=1}^{n}p(x_{i},y_{i})log_{2}p(x_{i}|y_{i}) = \sum_{j=1}^{n}p(y_{j})H(X|y_{j})$

In [1]:
# 计算
import math
entropy_2_2 = -1/2 * math.log(1/2, 2) - 1/2 * math.log(1/2, 2)
print('两个样本的概率均为1/2的熵：', entropy_2_2)

两个样本的概率均为1/2的熵： 1.0


回到ID3算法。   
<li style='color:red;size:12'>H(X)度量了X的不确定性;</li>
<li style='color:red;size:12'>条件熵H(X|Y)度量了我们在知道Y以后X剩下的不确定性;</li>

那么，H(X)-H(X|Y)呢？    
从上面的描述可以看出，它度量了X在知道Y以后不确定性减少的程度，这个度量我们在信息论中称为**互信息**，记为$I(X,Y)$。   

在决策树ID3算法中叫做**信息增益information gain**。   
ID3算法就是用信息增益来判断当前节点应该用什么特征来构建决策树。   
信息增益越大，越适合用来分类。   

用下面这个图很容易明白他们的关系。   
左边的椭圆代表H(X),右边的椭圆代表H(Y),中间重合的部分就是我们的互信息或者信息增益I(X,Y), 左边的椭圆去掉重合部分就是H(X|Y),右边的椭圆去掉重合部分就是H(Y|X)。两个椭圆的并就是H(X,Y)。

![title](../images/entropy_1.png)

## 2.决策树ID3算法的思路
ID3算法就是用信息增益大小来判断当前节点应该用什么特征来构建决策树的：  
用计算出的信息增益最大的特征来简历决策树的当前节点。   

这里我们举一个信息增益计算的具体的例子。     
比如我们有15个样本D，输出为0或者1。    
其中有9个输出为1， 6个输出为0。      
样本中有个特征A，取值为A1，A2和A3。  
在取值为A1的样本的输出中，有3个输出为1， 2个输出为0，   
取值为A2的样本输出中,2个输出为1,3个输出为0，   
在取值为A3的样本中，4个输出为1，1个输出为0.   

In [7]:
import math
# 样本D的熵为：
entropy_of_sample = -(9/15 * math.log(9/15, 2) + 6/15 * math.log(6/15, 2))

# 样本D在特征A下的条件熵为：
features_A1_entropy = -3/5 * math.log(3/5, 2) - 2/5 * math.log(2/5, 2)
features_A2_entropy = -2/5 * math.log(2/5, 2) - 3/5 * math.log(3/5, 2)
features_A3_entropy = -4/5 * math.log(4/5, 2) - 1/5 * math.log(1/5, 2)
features_A_conditional_entropy = 5/15 * features_A1_entropy + 5/15 * features_A2_entropy + 5/15 * features_A3_entropy

# 对应的信息增益，即 互信息
mutual_information_of_A = entropy_of_sample - features_A_conditional_entropy
print('Information Gain from Feature A：', mutual_information_of_A)

Information Gain from Feature A： 0.08300749985576883


### 具体的算法过程：
#### 输入：
    输入的是m个样本，样本输出集合为D，每个样本有n个离散特征，特征集合即为A，输出为决策树T。
    
#### 算法的过程为：
- 初始化信息增益的阈值$\epsilon$。
- 判断样本是否为同一类输出$D_{i}$，如果是则返回单节点树$T$。标记类别为$D_{i}$。
- 判断特征是否为空，如果是则返回单节点树$T$，标记类别为样本输出类别$D$实例数最多的类别。
- 计算$A$的信息增益小于阈值$\epsilon$，则返回单节点树$T$，标记类别为样本中输出类别$D$实例数最多的类别。
- 否则，按特征$A_{g}$的不同取值$A_{gi}$将对应的样本输出$D$分成不同的类别$D_{i}$。每个类别产生一个子节点。对应特征值为$A_{gi}$。返回增加了节点的树$T$。
- 对于所有的子节点，令$D = D_{i}$, $A=A-\{A_{g}\}$递归调用上面的2-6步，得到子树$T_{i}$并返回。 

    

以上的算法过程和李航的书上的表述基本类似，但是更好理解，符号和实例结合。

## 3.决策树ID3算法的不足
ID3算法虽然提出了新思路，但是还是又很多值得改进的地方。

- ID3没有考虑连续特征，比如长度，密度都是连续值，无法在ID3运用。这大大限制了ID3的用途。
- ID3采用信息增益大的特征优先建立决策树的节点。
    - 很快就被人发现，在相同条件下，取值比较多的特征比取值少的特征信息增益大。  
    - 比如：一个变量有2个值，各为1/2，另一个变量为3个值，各为1/3，其实他们都是完全不确定的变量，但是取3个值的比取2个值的信息增益大。
    - 这个问题需要校正，如何校正呢？
- ID3算法对于缺失值的情况没有做考虑
- 没有考虑过拟合的问题

基于ID3算法的以上不足，对ID3算法做了改进，这就是C4.5算法。
    

## 4.决策树C4.5算法的改进
ID3算法主要有四个不足：
    - 不能处理连续特征
    - 用信息增益作为标准容易偏向于取值较多的特征
    - 缺失值的情况未考虑
    - 过拟合问题

昆兰在C4.5算法中改进了上述4个问题。

### 4.1 ID3不能处理连续特征问题
对于第一个问题，ID3算法不能处理连续特征，C4.5的思路是将连续的特征离散化。  

对于m个样本的连续特征A有m个，从小到大排列为$a_{1}, a_{2}, ... a_{m}$，则C4.5取相邻两样本值的平均数，一共取得$m-1$个划分点，其中第$i$个划分点$T_{i}$表示为：  
$T_{i} = \frac{a_{i}+a{j}+1}{2}$。

对于这$m-1$个点，分别计算以该点作为二元分类点时的信息增益。   
选择信息增益最大的点作为该连续特征的二元离散分类点。   

比如取到的增益最大的点为$a_{t}$，则小于$a_{t}$的值为类别1，大于$a_{t}$的值为类别2，这样我们就做到了连续特征的离散化。    

要注意的是，与离散属性不同的是，如果当前节点为连续属性，则该属性后面还可以参与子节点的产生选择过程。  

### 4.2 ID3问题2：用信息增益作为标准容易偏向于取值较多的特征
对于第二个问题，信息增益作为标准容易偏向于取值较多的特征的问题。  
我们引入了一个**信息增益比**的变量$I_{R}(X,Y)$，它是信息增益和特征熵的比值。

信息增益比 = 信息增益 / 特征熵  

$I_{R}(D,A) = \frac{I(D,A)}{H_{A}{D}}$    

其中，D为样本特征输出的集合，A为样本特征，对于特征熵$H_{A}(D)$，表达式如下：
$H_{A}(D) = -\sum_{i=1}^{n}\frac{|D_{i}|}{|D|}log_{2}\frac{|D_{i}|}{|D|}$

其中，n为特征A的类别数，$D_{i}$为特征A的第$i$个取值对应的样本个数。D为样本个数。

**特征数越多的特征对应的特征熵越大， 它作为分母，可以小郑信息增益容易偏向于取值较多的特征的问题。**  


### 4.3 ID3问题3：缺失值处理
对于第三个缺失值处理的问题，主要需要解决的是两个问题，
- 一是在样本某些特征缺失的情况下选择划分的属性，
- 二是选定了划分属性，对于在该属性上缺失特征的样本的处理。  

对于第一个子问题，对于某一个有缺失特征值的特征A。   
C4.5的思路是将数据分成两部分，对每个样本设置一个权重（初始值可以都为1），然后划分数据，一部分是有特征值A的数据D1，另一部分是没有特征A的数据D2。   
然后对于没有缺失特征A的数据集D1来和对应的A特征的格格之一起计算加权重后的信息增益比。   
最后乘上一个系数，这个系数是无特征A缺失的样本加权后所占加权总样本的比例。  


对于第二个子问题，可以将缺失特征的样本同时划分入所有的子节点，不过将该样本的权重按各个子节点样本的数量比例来分配。  
比如缺失特征A的样本a之前的权重为1，特征A有三个特征值A1,A2,A3。  
3个特征值对应的无缺失A特征的样本个数为2,3,4。  
则a同时划分入A1,A2,A3。  
对应的权重调节为2/9, 3/9, 4/9。  

### 4.4 ID3问题4：过拟合问题
对于第4个问题，C4.5引入了正则化系数进行初步的剪枝。

### 总结：除了上面针对C4.5问题的解决方法，C4.5和ID3算法的思路区别不大。

## 5.决策树C4.5算法的不足与思考
C4.5虽然改进或者改善了ID3算法的几个主要的问题，仍然有优化的空间。  

- 由于决策树算法非常容易过拟合，因此对于生成的决策树必须要进行剪枝。 剪枝的算法有非常多，C4.5的剪枝方法有优化的空间。
    - 一种是预剪枝，即在生成决策树的时候就决定是否剪枝。
    - 另一个是后剪枝，即先生成决策树，再通过交叉验证来剪枝。
- C4.5生成的多叉树，即一个父节点可以有多个节点。很多时候，在计算机中二叉树模型会比多叉树运算效率高。
- C4.5只能用于分类，如果能将决策树用于回归的话可以扩大它的使用范围。
- C4.5由于使用了熵模型，里面有大量的耗时的对数运算，如果是连续值还有大量的排序运算。如果能够加以模型简化可以减少运算强度但又不牺牲太多准确性的话，那就更好了。

这4个问题在CART树里面部分加以了改进。   
如果不考虑集成学习的话，在普通的决策树算法里，CART算法是比较优的算法了。  
scikit-learn的决策树使用的也是CART算法。  

## 6.CART是ID3，C4.5的进化
上面我们讲到了决策树里ID3算法，和ID3算法的改进版C4.5算法。    
对于C4.5算法，我们也提到了它的不足，比如模型是用较为复杂的熵来度量，使用了相对较为复杂的多叉树，只能处理分类不能处理回归等。    
对于这些问题， CART算法大部分做了改进。    
CART算法也就是我们下面的重点了。 

由于CART算法可以做回归，也可以做分类，我们分别加以介绍。
先从**CART分类树**算法开始，重点比较和C4.5算法的不同点。
 
接着介绍**CART回归树算法**，重点介绍和CART分类树的不同点。  
然后我们讨论CART树的建树算法和剪枝算法，最后总结决策树算法的优缺点。

## 7.CART分类树算法的最优特征选择方法
我们知道，在ID3算法中我们使用了**信息增益**来选择特征，信息增益大的优先选择。  
在C4.5算法中，采用了**信息增益比**来选择特征，以减少信息增益容易选择特征值多的特征的问题。  
但无论是ID3还是C4.5，都是基于信息论的熵模型的，这里面会涉及大量的对数运算。  
能不能简化模型同时也不至于完全丢失熵模型的优点呢？ 有！    

CART分类树算法使用**基尼系数**来代替信息增益比，基尼系数代表了模型的不纯度，基尼系数越小，则不纯度越低，特征越好。  
这和信息增益（比）是相反的。    

具体的，在分类问题中，假设有K个类别，第k个类别的概率为$p_{k}$，则基尼系数的表达式为：
    $Gini(p) = \sum_{k=1}^{K}p_{k}(1-p_{k}) = 1 - \sum_{k=1}^{K}p_{k}^{2}$
    

如果是二类分类问题，计算就更加简单了，如果属于第一个样本输出的概率是p，则基尼系数的表达式为：
    $Gini(p) = 2p*(1-p)$

对于给定的样本D，假设有K个类别，第k个类别的数量为$C_{k}$，则样本D的基尼系数表达式为：
    $Gini(D) = 1-\sum_{k=}^{K}(\frac{|C_{k}|}{|D|})^{2}$     
    
特别地，对于样本D，如果根据特征A的某个值a，把D分成D1和D2两部分，则在特征A的条件下，D的基尼系数表达式为：   
    $Gini(D,A) = \frac{|D_{1}|}{|D|}Gini(D_{1}) + \frac{|D_{2}|}{|D|}Gini(D_{2})$    
    
大家可以比较下基尼系数表达式和熵模型的表达式，二次运算是不是比对数简单很多？    
尤其是二类分类的计算，更加简单。  
但是简单归简单，和熵模型的度量方式比，基尼系数对应的误差有多大呢？       
对于二元分类，基尼系数和熵之间的曲线如下：

![title](../images/gini_entropy.png)

从上图可以看出，基尼系数和熵之半的曲线非常接近，仅仅在45度角附近误差稍大。    
因此，基尼系数可以作为熵模型的一个近似替代。   
而CART分类树算法就是使用的**基尼系数**来选择决策树的特征。   
同时，为了进一步简化，CART分类树算法每次仅仅对某个特征的值进行二分，而不是多分，这样CART分类树算法建立起来的是二叉树，而不是多叉树。   
好处：  
- 可以进一步简化基尼系数的计算；
- 可以建立一个更加优雅的二叉树模型。

## 8.CART分类树算法对于连续特征和离散特征处理的改进
对于CART分类树连续值的处理问题，其思想和C4.5是相同的，都是将连续的值特征离散化。   
唯一的区别在于选择划分点时的度量方式不同，C4.5使用的是信息增益比，而CART分类树使用的是基尼系数。  

具体的思路如下：   
比如m个样本的连续特征A有m个，从小打到排列为$a_{1},a_{2},...,a_{m}$，则CART算法取相邻两样本值的中位数，一共取得m-1个划分点，其中第$i$个划分点$T_{i}$表示为：    
$T_{i} = \frac{a_{i}+a_{j} +1}{2}$     
对于这m-1个点，分别计算以该点作为二元分类点时的基尼系数。   
选择基尼系数最小的点作为该连续特征的二元离散分类点。   
比如取到的基尼系数最小的点为$a_{t}$，则小于$a_{t}$的值为类别1，大于$a_{t}$的值为类别2，这样我们就做到了连续特征的离散化。   
要注意的是，与离散属性不同的是，如果当前节点为连续属性，则该属性后面还可以参与子节点的产生选择过程。      


对于CART分类树离散值的处理问题，采用的思路是不停的二分离散特征。    
回忆下ID3或者C4.5，如果某个特征A被选取建立决策树节点，如果它有$A_{1},A_{2},A_{3}$三种类别，我们会在决策树上一下建立一个三叉的节点。这样导致决策树是多叉树。    
但是，CART分类树使用的方法不同，它采用的是不停的二分。   
还是这个例子，CART分类树会考虑把A分成
      $\{A_{1}\}$和$\{A_{2}, A_{3}\}$，
      $\{A_{2}\}$和$\{A_{1}, A_{3}\}$， 
      $\{A_{3}\}$和$\{A_{1}, A_{2}\}$
    三种情况，找到基尼系数最小的组合，比如$\{A_{2}\}$和$\{A_{1}, A_{3}\}$，然后建立二叉树节点，一个节点是$A_{2}$对应的样本，另一个节点是$\{A_{1}, A_{3}\}$对应的节点。    
    同时，由于这次没有把特征A的取值完全分开，后面我们还有机会在子节点继续选择到特征A来划分$A_{1}$和$A_{3}$。   
    这和ID3算法，C4.5算法不同，在ID3或者C4.5算法的一棵子树中，离散特征只会参与一次节点的建立。  

## 9.CART分类树建立算法的具体流程
CART分类树建立算法的具体流程，之所以加上“建立”二字，是因为CART树算法还有独立的剪枝算法。   

算法输入是训练集D，基尼系数的阈值，样本个数阈值。（对应sklearn.tree.DecisionTreeClassifier类的参数的criterion='gini', min_samples_split=2)

输出的是决策树T。   

我们的算法从根节点开始，用训练集递归的建立CART树。  

- 对于当前节点的数据集D，如果样本个数小于阈值或者没有特征，则返回决策子树，当前节点停止递归。
- 计算样本集D的基尼系数，如果基尼系数大于阈值，则返回决策树子树，当前节点停止递归。   
- 计算当前节点现有的各个特征的各个特征值对数据集D的基尼系数，
    - 对于离散值和连续值的处理方法和基尼系数的计算见上文。  
    - 缺失值的处理方法和C4.5算法里的处理方式相同。
- 在计算出来的各个特征的各个特征值对数据集D的基尼系数中，选择基尼系数最小的特征A和对应的特征值a。根据这个最优特征和最优特征值，把数据集划分为两部分D1和D2，同时建立当前节点的左右节点，左节点的数据集D为D1，右节点的数据集D为D2。
- 对左右的子节点递归的调用上面4步，生成决策树。  


对于生成的决策树做预测的时候，假如测试集的样本A落到了某个叶子节点，而节点里有多个训练样本。   
则对于A类别预测采用的是这个叶子节点里概率最大的类别。  

## 10.CART回归树建立算法
CART回归树和CART分类树的建立算法大部分是类似的，所以我们这个只讨论CART回归树和CART分类树的建立算法不同的地方。   

首先，我们要明白，什么是回归树，什么是分类树。   
两者的区别在于样本输出，如果样本输出是离散值，那么这个是一棵分类树。   
如果样本输出是连续值，那么这是一棵回归树。  

除了概念的不同，CART回归树和CART分类树的建立和预测的区别主要有下面两点：
- 连续值的处理方法不同
- 决策树建立后做预测的方式不同

对于连续值的处理，我们知道CART分类树采用的是用基尼系数的大小来度量特征的各个划分点的优劣情况。  
这比较适合分类模型，但是对于回归模型，我们使用了常见的**和方差**的度量方式，CART回归树的度量目标是，**对于任意划分特征A，对于的任意划分点s两边划分成的数据集D1和D2，求出使D1和D2各自集合的均方差最小，同时D1和D2的均方差之和最小所对应的特征和特征值划分点。**

表达式为：   